## KoT5 모델 테스트

In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install torch transformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# KoT5 모델과 토크나이저 로드
model_name = "lcw99/kobart-korean-text-summary"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 질문과 답변 설정
question = "포스코 DX에 입사하게 된 이유와 관심을 가지게 된 배경을 간단히 설명해 주세요."
answer = """
[리딩 철강 브랜드의 변화]
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며 POSCO에 대한 관심을 가졌습니다. 
그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX를 알게 되었습니다.
"""

# 입력 텍스트 구성 (간단한 요약 유도)
input_text = f"{question} {answer}"

# 토큰화 및 요약 생성
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=15,  # 제목 길이 제한
    min_length=5,   # 짧고 간결한 요약
    num_beams=4,
    no_repeat_ngram_size=3,
    early_stopping=True
)

# 생성된 제목 요약
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("생성된 제목:", summary)


OSError: lcw99/kobart-korean-text-summary is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# T5 모델과 토크나이저 로드
model_name = "KETI-AIR/ke-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 요약할 텍스트
text = """
[리딩 철강 브랜드의 변화]
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며, POSCO에 대한 관심을 가졌습니다. 그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 또한 글로벌 기업의 기술력이 아닌 사내 고유의 인프라를 활용해 신기술을 적용했기 때문에 확장성과 안정성을 동시에 갖출 수 있었습니다.
"""

# 프롬프트 추가와 토큰화
prompt_text = "이 텍스트를 한 줄로 요약하세요: " + text
inputs = tokenizer(prompt_text, return_tensors="pt", max_length=512, truncation=True)

# 요약 생성
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=20,
    min_length=5,
    num_beams=5,
    length_penalty=1.5,
    early_stopping=True
)

# 생성된 요약문 디코딩
headline = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("한줄 요약 (개조식):", " •", headline)


한줄 요약 (개조식):  • 原原原原原原原原原原原原原原原原原原原
